In [ ]:
!pip3 install pysd

In [21]:
import os
from pysd import read_vensim
import pandas as pd
import re


In [23]:
def parse_mdl_file(filepath):
    variables = []
    connections = []
    variable_ids = set()  # Para evitar duplicados

    with open(filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split(',')

        if not parts or len(parts) < 5:
            continue

        prefix = parts[0]

        if prefix in ['10', '11', '12']:  # Variable
            try:
                tipo = {
                    '10': 'auxiliary',
                    '11': 'stock',
                    '12': 'flow'
                }[prefix]

                vid = f"v{parts[1]}"
                name = parts[2].strip('"')

                if name == '0' or name.strip() == '':
                    continue  # Ignorar nombres inválidos

                if vid in variable_ids:
                    continue  # Ignorar duplicados

                x = int(parts[3])
                y = int(parts[4])

                variables.append({
                    "id": vid,
                    "name": name,
                    "type": tipo,
                    "x": x,
                    "y": y
                })

                variable_ids.add(vid)

            except Exception as e:
                print("[ERROR variable] Línea inválida:", line.strip(), "→", e)
                continue

        elif prefix == '1':  # Conexión
            try:
                cid = f"c{parts[1]}"
                from_id = f"v{parts[2]}"
                to_id = f"v{parts[3]}"

                text = line.strip()

                # Extraer color (primer patrón RGB)
                color_match = re.search(r"(\d+-\d+-\d+)", text)
                color = color_match.group(1) if color_match else None

                # Determinar polaridad según color
                if color == "0-0-255":
                    polarity = "positive"
                elif color == "255-0-0":
                    polarity = "negative"
                else:
                    polarity = "positive"

                # Extraer coordenadas de curva
                coord_match = re.search(r"\((\d+),(\d+)\)", text)
                if coord_match:
                    x_curve = int(coord_match.group(1))
                    y_curve = int(coord_match.group(2))
                else:
                    x_curve = None
                    y_curve = None

                connections.append({
                    "id": cid,
                    "from": from_id,
                    "to": to_id,
                    "color": color,
                    "polarity": polarity,
                    "x_curve": x_curve,
                    "y_curve": y_curve
                })

            except Exception as e:
                print("[ERROR conexión] Línea inválida:", line.strip(), "→", e)
                continue

    return {
        "name": os.path.basename(filepath),
        "variables": variables,
        "connections": connections
    }

In [26]:
parse_mdl_file("eficiencia-movilidad-causal.mdl")

{'name': 'eficiencia-movilidad-causal.mdl',
 'variables': [{'id': 'v3',
   'name': 'Congestión vehicular',
   'type': 'auxiliary',
   'x': 1301,
   'y': 256},
  {'id': 'v4',
   'name': 'Eficiencia de movilidad',
   'type': 'auxiliary',
   'x': 1168,
   'y': 467},
  {'id': 'v6',
   'name': 'Fluidez del tráfico',
   'type': 'auxiliary',
   'x': 880,
   'y': 445},
  {'id': 'v8',
   'name': 'Uso del transporte público',
   'type': 'auxiliary',
   'x': 905,
   'y': 167},
  {'id': 'v9',
   'name': 'Número de vehículos en circulación',
   'type': 'auxiliary',
   'x': 1040,
   'y': 256},
  {'id': 'v12',
   'name': 'Calidad de transporte público',
   'type': 'auxiliary',
   'x': 733,
   'y': 117},
  {'id': 'v14',
   'name': 'Extorsionesa a transportistas',
   'type': 'auxiliary',
   'x': 562,
   'y': -8},
  {'id': 'v15',
   'name': 'Nivel de educación vial',
   'type': 'auxiliary',
   'x': 1565,
   'y': -31},
  {'id': 'v16',
   'name': 'Total de Infracciones',
   'type': 'auxiliary',
   'x': 12

In [27]:
parse_mdl_file("frecuencia-de-mantenimiento-forrester.mdl")

{'name': 'frecuencia-de-mantenimiento-forrester.mdl',
 'variables': [{'id': 'v2',
   'name': 'Disponibilidad de la flota',
   'type': 'auxiliary',
   'x': 1137,
   'y': 415},
  {'id': 'v3', 'name': '48', 'type': 'flow', 'x': 839, 'y': 408},
  {'id': 'v7',
   'name': 'Vehiculos que regresan de mantenimiento',
   'type': 'auxiliary',
   'x': 957,
   'y': 446},
  {'id': 'v8', 'name': '48', 'type': 'flow', 'x': 1423, 'y': 415},
  {'id': 'v12',
   'name': 'Vehiculos que se retiran por mantenimiento',
   'type': 'auxiliary',
   'x': 1314,
   'y': 457},
  {'id': 'v13',
   'name': 'Cantidad de vehiculos en operacion',
   'type': 'auxiliary',
   'x': 1350,
   'y': 654},
  {'id': 'v17',
   'name': 'Vehiculos asignados a operar',
   'type': 'auxiliary',
   'x': 1207,
   'y': 577},
  {'id': 'v18', 'name': '48', 'type': 'flow', 'x': 1710, 'y': 661},
  {'id': 'v22',
   'name': 'Vehiculos devueltos al deposito',
   'type': 'auxiliary',
   'x': 1587,
   'y': 695},
  {'id': 'v23',
   'name': 'Cantidad 

In [10]:
model = read_vensim('frecuencia-de-mantenimiento-forrester.mdl')

In [17]:
data = model.run()

In [18]:
data

,FINAL TIME,INITIAL TIME,SAVEPER,TIME STEP,"""Error/Faltante3""","""Error/Faltante4""",Cantidad de plazas apartadas por mantenimiento,Objetivo1,Objetivo2,Cantidad de reparaciones preventivas,...,Tasa de liberacion de plazas,Tasa de retorno al deposito,Tiempo promedio de mantenimiento,Total de la flota,Total de plazas,Vehiculos asignados a operar,Vehiculos devueltos al deposito,Vehiculos operativos que mostraron fallas,Vehiculos que regresan de mantenimiento,Vehiculos que se retiran por mantenimiento
time,,,,,,,,,,,,,,,,,,,,,
2025,2036,2025,1,1,25.000000,-20.0,8.0,9,15,24,...,1.0,0.1,2,50,40,0.000000,3.000000,9,25.0,8.82
2026,2036,2025,1,1,28.000000,-12.0,7.0,9,15,24,...,1.0,0.1,2,50,40,11.326000,2.700000,8,16.5,7.84
2027,2036,2025,1,1,19.374000,-13.0,8.0,9,15,24,...,1.0,0.1,2,50,40,9.459800,3.562600,10,18.0,9.80
2028,2036,2025,1,1,13.476800,-12.0,11.0,9,15,32,...,1.0,0.1,2,50,40,8.577940,4.152320,12,18.5,11.76
2029,2036,2025,1,1,9.051180,-9.0,12.0,9,15,40,...,1.0,0.1,2,50,40,7.291382,4.594882,13,19.5,12.74
2030,2036,2025,1,1,6.354680,-8.0,13.0,9,15,40,...,1.0,0.1,2,50,40,6.919415,4.864532,14,20.0,13.72
2031,2036,2025,1,1,4.299797,-7.0,13.0,9,15,40,...,1.0,0.1,2,50,40,6.471824,5.070020,15,20.0,14.70
2032,2036,2025,1,1,2.897993,-7.0,13.0,9,15,40,...,1.0,0.1,2,50,40,5.651547,5.210201,15,20.5,14.70
2033,2036,2025,1,1,2.456647,-7.0,13.0,9,15,40,...,1.0,0.1,2,50,40,5.755464,5.254335,15,20.5,14.70
